In [133]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [301]:
import math
import string
import re

In [137]:
def missing_zero_values_table(df):
    zero_val = (df == 0.00).astype(int).sum(axis=0)
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
    mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
    mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
    mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
    mz_table['Data Type'] = df.dtypes
    mz_table = mz_table[
    mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) + " columns that have missing values.")
    
    return mz_table

In [2]:
videogames = pd.read_json('Data/meta_Video_Games.json', lines=True)
reviews = pd.read_json('Data/Video_Games.json', lines=True)

## Cleaning Video Games DF

In [45]:
#drop also_buy, also_view, date, details, similar_item, tech1, tech2
main_videogames = videogames.copy()
main_videogames.drop(columns=['also_buy', 'also_view', 'date', 'details', 'similar_item', 'tech1', 'tech2'], inplace=True)

In [170]:
#Check for missing values #1
missing_zero_values_table(main_videogames)

Your selected dataframe has 10 columns and 84893 Rows.
There are 9 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
price,0,69338,81.7,69338,81.7,object
feature,0,22961,27.0,22961,27.0,object
image,0,22889,27.0,22889,27.0,object
description,0,10170,12.0,10170,12.0,object
rank,0,4510,5.3,4510,5.3,object
brand,0,4030,4.7,4030,4.7,object
category,0,2426,2.9,2426,2.9,object
main_cat,0,166,0.2,166,0.2,object
title,0,14,0.0,14,0.0,object


### Changing missing title names to their actual titles

In [184]:
indexes = [467, 11873, 11876, 11878, 13376, 24782, 24785, 24787, 29674, 38379, 52373, 61916, 63263, 64453]
title_names = ['Dune 2000', 'Virtual Stick Pro', 'Saturn Control Pad Mist Gray', 'Saturn control pad', 'Dune 2000', 
               'Virtual Stick Pro', 'Saturn Control Pad Mist Gray', 'Saturn control pad', 'Sega Vertual Stick', 'Puppet Motel',
               'Friendship Collection New Life','Ranch Story Connected New World', 'Kirby Triple Deluxe', 'Detective Conan Phantom Rhapsody']

In [191]:
for idx, val in enumerate(indexes):
    main_videogames.loc[val, 'title'] = title_names[idx]

In [192]:
#Check for missing values #2
missing_zero_values_table(main_videogames)

Your selected dataframe has 10 columns and 84893 Rows.
There are 8 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
price,0,69338,81.7,69338,81.7,object
feature,0,22961,27.0,22961,27.0,object
image,0,22889,27.0,22889,27.0,object
description,0,10170,12.0,10170,12.0,object
rank,0,4510,5.3,4510,5.3,object
brand,0,4030,4.7,4030,4.7,object
category,0,2426,2.9,2426,2.9,object
main_cat,0,166,0.2,166,0.2,object


In [233]:
float_puncs = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'
str1=""

def remove_puncs_into_floats(item):
    table = str.maketrans('','', float_puncs)
    stripped = [i.translate(table) for i in item]
    return float(str1.join(stripped))

#change missing prices to -1, and change string prices to floats
prices = main_videogames.price.apply(lambda x: float(-1) if isinstance(x, float) else remove_puncs_into_floats(x))

In [234]:
#Since 81.7% of prices are missing, and not doing a recommender based on price, remove price column.
#Since images are not being used to create a recommender, remove image column.
main_videogames.drop(columns=['price','image'],inplace=True)

In [235]:
#Check for missing values #3
missing_zero_values_table(main_videogames)

Your selected dataframe has 8 columns and 84893 Rows.
There are 6 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
feature,0,22961,27.0,22961,27.0,object
description,0,10170,12.0,10170,12.0,object
rank,0,4510,5.3,4510,5.3,object
brand,0,4030,4.7,4030,4.7,object
category,0,2426,2.9,2426,2.9,object
main_cat,0,166,0.2,166,0.2,object


In [328]:
def remove_puncs(item):
    table = str.maketrans('','', float_puncs)
    stripped = [i.translate(table) for i in item]
    return str1.join(stripped)

In [338]:
#converting rank strings to just rank numbers
rank = main_videogames["rank"].apply(lambda x: "999999999" if isinstance(x, float) else remove_puncs(x).split(" ",1)[0])

In [339]:
rank = rank.apply(lambda x: x.split("i", 1)[0])

In [344]:
#find messed up values in rnak #4
for idx, val in enumerate(rank):
    if val == 'productDescr':
        print(idx)

52316


In [345]:
rank[52316] = 999999999

In [346]:
rank = rank.apply(lambda x: float(x))

In [348]:
main_videogames['rank']=rank

In [349]:
#Check for missing values #4
missing_zero_values_table(main_videogames)

Your selected dataframe has 8 columns and 84893 Rows.
There are 5 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
feature,0,22961,27.0,22961,27.0,object
description,0,10170,12.0,10170,12.0,object
brand,0,4030,4.7,4030,4.7,object
category,0,2426,2.9,2426,2.9,object
main_cat,0,166,0.2,166,0.2,object


In [369]:
#Change all nulls in main_cat into 'Video Games'
main_cat = main_videogames['main_cat'].apply(lambda x: 'Video Games' if isinstance(x, float) else x)
main_videogames['main_cat'] = main_cat

#Change all nulls in features into 'no_features'
features = main_videogames['feature'].apply(lambda x: 'no_features' if isinstance(x, float) else x)
main_videogames['feature'] = features

#Change all null in description into 'no_description'
description = main_videogames['description'].apply(lambda x: 'no_description' if isinstance(x, float) else x)
main_videogames['description'] = description

In [370]:
#Check for missing values #5
missing_zero_values_table(main_videogames)

Your selected dataframe has 8 columns and 84893 Rows.
There are 2 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
brand,0,4030,4.7,4030,4.7,object
category,0,2426,2.9,2426,2.9,object


In [388]:
#Change null brands to 'no_brand'
brand = main_videogames['brand'].apply(lambda x: 'no_brand' if isinstance(x, float) else x)

#Remove html taggings "by\n \n"
filter_tag = 'by\n    \n    '
brand = brand.apply(lambda x: x[12:] if filter_tag in x else x)
main_videogames['brand'] = brand

#Change null category to ['Video Games']
category = main_videogames['category'].apply(lambda x: ['Video Games'] if isinstance(x, float) else x)
main_videogames['category'] = category

In [389]:
#Check for missing values #6
missing_zero_values_table(main_videogames)

Your selected dataframe has 8 columns and 84893 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type


In [420]:
#Change category into a full string
str2 = " "
category = main_videogames['category'].apply(lambda x: str2.join(x))
main_videogames['category'] = category

#Change description into a full string
description = main_videogames['description'].apply(lambda x: str2.join(x) if x!='no_description' else x)
main_videogames['description'] = description

,asin,brand,category,description,feature,main_cat,rank,title
0,0042000742,Fidelity Electronics,Video Games PC Games,no_description,no_features,Toys & Games,2623937.0,Reversi Sensory Challenger
1,0078764343,EA Games,Video Games Xbox 360 Games,Brand new sealed!,no_features,Video Games,67231.0,Medal of Honor: Warfighter - Includes Battlefi...
2,0276425316,Nintendo,Video Games Retro Gaming & Microconsoles Super...,no_description,no_features,Video Games,134433.0,street fighter 2 II turbo super nintendo snes ...
3,0324411812,MAS SYSTEMS,Video Games Xbox 360 Accessories Controllers J...,MAS's Pro Xbox 360 Stick (Perfect 360 Stick) S...,[Original PCB used from Xbox 360 Control Pad (...,Video Games,105263.0,Xbox 360 MAS STICK
4,0439335310,Advanced Software Pty. Ltd.,Video Games PC Games Grades 2-12 Spelling Pro...,"Phonics Alive! 3, The Speller teaches students...","[Grades 2-12, Spelling Program, Teaches Spelli...",Video Games,92397.0,Phonics Alive! 3: The Speller


# Cleaning Reviews DF

In [116]:
#change image to 0 or 1
main_reviews = reviews.copy()
image = [0 if isinstance(i,float) else 1 for i in main_reviews.image]
main_reviews['image'] = image

In [128]:
#drop columns = reviewTime, reviewName, style, vote
main_reviews.drop(columns=['reviewTime', 'reviewerName', 'style', 'vote'], inplace=True)

### Handle Missing

In [138]:
missing_zero_values_table(main_reviews)

Your selected dataframe has 8 columns and 2565349 Rows.
There are 2 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
reviewText,0,1715,0.1,1715,0.1,object
summary,0,811,0.0,811,0.0,object


In [144]:
np.unique(reviews.reviewText.isna(), return_counts=True)

(array([False,  True]), array([2563634,    1715]))

In [150]:
np.unique(reviews.summary.isna(), return_counts=True)

(array([False,  True]), array([2564538,     811]))

In [162]:
reviewText = main_reviews.reviewText.apply(lambda x: 'no_text_was_given' if isinstance(x, float) else x)

In [166]:
summary = main_reviews.summary.apply(lambda x: 'no_summary_given' if isinstance(x, float) else x)

In [168]:
main_reviews['reviewText'] = reviewText
main_reviews['summary'] = summary

In [169]:
missing_zero_values_table(main_reviews)

Your selected dataframe has 8 columns and 2565349 Rows.
There are 0 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type


##### Done Cleaning Reviews DF